In [15]:
from tenacity import retry, stop_after_attempt

x = 0

@retry(stop=stop_after_attempt(1))
def stop_after_7_attempts():
    global x
    if x == 2:
        return x
    print("Stopping after 7 attempts", x)
    x += 1
    raise Exception

In [16]:
stop_after_7_attempts()

Stopping after 7 attempts 0


RetryError: RetryError[<Future at 0x7f4f74525240 state=finished raised Exception>]

In [18]:
labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

target = {'a', 'b', 'c'}
pred_1 = {'a', 'b'}
pred_2 = {'a', 'b', 'c', 'd'}
pred_3 = {'c', 'd', 'e'}
pred_4 = {'d', 'e', 'f'}
pred_5 = {'a', 'b', 'c'}

def jaccard_similarity(list1, list2):
    intersection = len(list1.intersection(list2))
    union = len(list1.union(list2))
    return intersection / union

print(jaccard_similarity(target, pred_1))
print(jaccard_similarity(target, pred_2))
print(jaccard_similarity(target, pred_3))
print(jaccard_similarity(target, pred_4))
print(jaccard_similarity(target, pred_5))


0.6666666666666666
0.75
0.2
0.0
1.0


In [19]:
def accuracy(target, pred):
    return len(target.intersection(pred)) / len(target)

print(accuracy(target, pred_1))

0.6666666666666666


In [27]:

multilabel_classes = ['lists_createoradd', 'calendar_query', 'email_sendemail', 'news_query',
 'play_music', 'play_radio', 'qa_maths', 'email_query','weather_query',
 'calendar_set','iot_hue_lightdim','takeaway_query','social_post'
 'email_querycontact','qa_factoid','calendar_remove','cooking_recipe',
 'lists_query','general_quirky','alarm_query','takeaway_order',
 'iot_hue_lightup','lists_remove','qa_currency','play_game',
 'play_audiobook','qa_definition','music_query','datetime_query',
 'transport_query','iot_hue_lightoff','iot_hue_lightchange',
 'iot_hue_lighton','alarm_set','music_likeness','recommendation_movies',
 'transport_ticket','recommendation_locations','audio_volume_mute',
 'iot_wemo_on','play_podcasts','datetime_convert','audio_volume_other',
 'recommendation_events','alarm_remove','iot_coffee','music_dislikeness',
 'general_joke','social_query']

len(multilabel_classes)

48

In [24]:
# calculate tp, fp, fn, tn

def confusion_matrix(target, pred):
    tp = len(target.intersection(pred))
    fp = len(pred - target)
    fn = len(target - pred)
    if n_labels:
        tn = len(labels) - (tp + fp + fn)
    else:
        tn = None
    return tp, fp, fn, tn

print(confusion_matrix(target, pred_1))
print(confusion_matrix(target, pred_2))
print(confusion_matrix(target, pred_3))
print(confusion_matrix(target, pred_4))
print(confusion_matrix(target, pred_5))

(2, 0, 1, None)
(3, 1, 0, None)
(1, 2, 2, None)
(0, 3, 3, None)
(3, 0, 0, None)


In [26]:
def f1_score(target, pred):
    tp, fp, fn, tn = confusion_matrix(target, pred)
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f1_score(target, pred_1))
print(f1_score(target, pred_2))
print(f1_score(target, pred_3))
print(f1_score(target, pred_4))
print(f1_score(target, pred_5))

0.8
0.8571428571428571
0.3333333333333333
0
1.0


In [42]:
x = [{'lists_createoradd', 'calendar_query', 'email_sendemail'}, {'lists_createoradd', 'calendar_query', 'email_sendemail'}, {'lists_createoradd', 'calendar_query', 'email_sendemail'}, {'calendar_query', 'email_sendemail'}, {'lists_createoradd', 'calendar_query', 'email_sendemail'}, {'calendar_query', 'email_sendemail'}, {'lists_createoradd', 'calendar_query', 'email_sendemail'}, {'calendar_query', 'email_sendemail'}, {'lists_createoradd', 'calendar_query', 'email_sendemail'}, {'lists_createoradd', 'calendar_query', 'email_sendemail'}]
y = {'lists_createoradd', 'calendar_query', 'email_sendemail'}

def get_confusion_matrix(y_true, y_pred, n_labels=0):
    tp = len(y_true.intersection(y_pred))
    fp = len(y_pred - y_true)
    fn = len(y_true - y_pred)
    if n_labels:
        tn = n_labels - (tp + fp + fn)
    else:
        tn = None
    return tp, fp, fn, tn

def get_precsion_recall_f1_score_accuracy(tp, fp, fn, tn):
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if tn else 0
    return precision, recall, f1, accuracy

def get_metrics(y_true, y_pred, n_labels=0):
    tp, fp, fn, tn = get_confusion_matrix(y_true, y_pred, n_labels)
    return get_precsion_recall_f1_score_accuracy(tp, fp, fn, tn)

def get_multilabel_classification_metrics(target, preds):
    tp, fp, fn, tn = 0, 0, 0, 0
    for pred in preds:
        tp += len(target.intersection(pred))
        fp += len(pred - target)
        fn += len(target - pred)
        tn += 48 - (tp + fp + fn)
    p = tp / (tp + fp) if (tp + fp) != 0 else 0
    r = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1 = 2 * (p * r) / (p + r) if (p + r) != 0 else 0
    a = (tp + tn) / (tp + tn + fp + fn) if tn else 0
    return {'precision': p, 'recall': r, 'f1': f1, 'accuracy': a}

from numpy import mean

print(get_metrics(y, x[0]))

print(get_multilabel_classification_metrics(set()y, x))

print(mean([get_metrics(y, x[i])[2] for i in range(len(x))]))

accurate = 0
for y_pred in x:
    if y_pred == y:
        accurate += 1
print(accurate / len(x))



(1.0, 1.0, 1.0, 0)
(1.0, 0.9, 0.9473684210526316, 0.991304347826087)
0.9400000000000001
0.7


In [29]:
type(y)

1.0

set